# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121709e+02     1.499945e+00
 * time: 0.7726540565490723
     1     1.096259e+01     8.165785e-01
 * time: 2.059311866760254
     2    -1.176505e+01     9.404673e-01
 * time: 2.659960985183716
     3    -3.398454e+01     7.812956e-01
 * time: 3.5493788719177246
     4    -4.742439e+01     5.946219e-01
 * time: 4.454025983810425
     5    -5.673524e+01     2.531876e-01
 * time: 5.325041055679321
     6    -5.956936e+01     2.330382e-01
 * time: 5.913683891296387
     7    -6.076760e+01     1.072037e-01
 * time: 6.507261037826538
     8    -6.133075e+01     4.034690e-02
 * time: 7.115097999572754
     9    -6.164173e+01     3.698958e-02
 * time: 7.71033787727356
    10    -6.182025e+01     4.084489e-02
 * time: 8.306405067443848
    11    -6.194211e+01     2.396300e-02
 * time: 8.895199060440063
    12    -6.202309e+01     1.940550e-02
 * time: 9.497601985931396
    13    -6.208795e+01     1.643955e-02
 * time: 10.087964057922363
    1

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671062
    AtomicLocal         -18.8557645
    AtomicNonlocal      14.8522627
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485358 
    Xc                  -19.3336815

    total               -62.261666457776